In this notebook, we compute ellipsoidal controlled invariant sets using the method proposed in [LB18].

[LB18] B. Legat, P. Tabuada, R. M. Jungers. Computing controlled invariant sets for hybrid systems with applications to model-predictive control, IFAC-PapersOnLine, Volume 51, Issue 16, 2018, Pages 193-198, ISSN 2405-8963.

The original code of [LB18] can be found in the following [github repository](https://github.com/blegat/SwitchOnSafety.jl).

We begin by formulating the problem.

In [2]:
# Load dependencies:
using MosekTools
using JuMP
using Polyhedra
using MathematicalSystems
using SwitchOnSafety
using LinearAlgebra

# Choose solver:
solver = with_optimizer(Mosek.Optimizer, QUIET=true)

# Vehicle parameters
C_af = 133000
C_ar = 98800
m = 1650
u = 30
b = 1.59
Iz = 2315
a = 1.11
steer_max = pi/2 
steer_min = -pi/2

# Continuous dynamics:
Ao = [  0                      1 u                            0
        0     -(C_af+C_ar)/(m*u) 0    (b*C_ar - a*C_af)/(m*u)-u
        0                      0 0                            1
        0 (b*C_ar-a*C_af)/(Iz*u) 0 -(a^2*C_af + b^2*C_ar)/(Iz*u)]

Bo = reshape([0, C_af/m, 0, a*C_af/Iz],4,1)

# Discretize using forward Euler method:
Ts = 0.1

A = Diagonal([1,1,1,1]) + Ao*Ts
B = Bo*Ts


# Safe Set:
G =[ 1     0     0     0
    -1     0     0     0
     0     1     0     0
     0    -1     0     0
     0     0     1     0
     0     0    -1     0
     0     0     0     1
     0     0     0    -1]
F = [0.9
     0.9
     1.2
     1.2
     0.05
     0.05
     0.3
     0.3]
safe_set = polyhedron(hrep(G, F), DefaultLibrary{Float64}(solver))

# Input Constraints:
input_set = polyhedron(convexhull([steer_min], [steer_max]))

system = ConstrainedLinearControlDiscreteSystem(A,B,safe_set,input_set)

┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/MosekTools/UJwlm.ji for MosekTools [1ec41992-ff65-5c91-ac43-2df89e9693a4]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/JuMP/DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/Polyhedra/17i4E.ji for Polyhedra [67491407-f73d-577b-9b50-8179a7c68029]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/MathematicalSystems/6oLdk.ji for MathematicalSystems [d14a8603-c872-5ed3-9ece-53e0e82e39da]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/j10/.julia/compiled/v1.0/SwitchOnSafety/EUPLd.ji for SwitchOnSafety [ceb7f16a-07bf-5f4a-9354-b68f01b1610f]
└ @ Base loading.jl:1190


ConstrainedLinearControlDiscreteSystem{Float64,Array{Float64,2},Array{Float64,2},DefaultPolyhedron{Float64,MixedMatHRep{Float64,Array{Float64,2}},MixedMatVRep{Float64,Array{Float64,2}}},Interval{Float64,StaticArrays.SArray{Tuple{1},Float64,1,1},StaticArrays.Size{(1,)}}}([1.0 0.1 3.0 0.0; 0.0 0.531717 0.0 -2.98088; 0.0 0.0 1.0 0.1; 0.0 0.0136242 0.0 0.404398], [0.0; 8.06061; 0.0; 6.37711], HalfSpace([1.0, 0.0, 0.0, 0.0], 0.9) ∩ HalfSpace([-1.0, 0.0, 0.0, 0.0], 0.9) ∩ HalfSpace([0.0, 1.0, 0.0, 0.0], 1.2) ∩ HalfSpace([0.0, -1.0, 0.0, 0.0], 1.2) ∩ HalfSpace([0.0, 0.0, 1.0, 0.0], 0.05) ∩ HalfSpace([0.0, 0.0, -1.0, 0.0], 0.05) ∩ HalfSpace([0.0, 0.0, 0.0, 1.0], 0.3) ∩ HalfSpace([0.0, 0.0, 0.0, -1.0], 0.3), HalfSpace([1.0], 1.5707963267948966) ∩ HalfSpace([-1.0], 1.5707963267948966) : convexhull([1.5708], [-1.5708]))

Compute the invariant set by searching for any ellipsoid with a given point in its interior.
As the system is reformulated into an algebraic system with safe set `safe_set * input_set`, the Chebyshev center is `(cheby_center, 0)`.

To avoid having to solve Bilinear Matrix Inequalities, we set the S-procedure scaling to `1.01` (found by a few trials, checking what gives the best `objective_value`).

In [89]:
# Compute controlled invariant maximum volume ellipsoid inside the safe set:

# Begin timing here:
@time begin
    # S-procedure constant:
    S_procedure_scaling = 1.01
    # Center of ellipsoid:
    cheby_center, cheby_radius = chebyshevcenter(safe_set, solver)
    cntr = [cheby_center; 0.0]
    # cntr = [0.0;0.0;0.0]

    using SwitchOnSafety
    variable = Ellipsoid(point = SetProg.InteriorPoint(cntr))
    max_vol_ell = invariant_set(system, solver, variable, λ = S_procedure_scaling)
end

show(max_vol_ell)

MOI.get(model, MOI.SolveTime()) = 0.014314889907836914
JuMP.termination_status(model) = OPTIMAL::TerminationStatusCode = 1
JuMP.primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
JuMP.objective_value(model) = 2.801672109673336e-10
  0.048703 seconds (47.09 k allocations: 3.273 MiB, 19.48% gc time)
SetProg.Sets.Translation{SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}},Float64,Array{Float64,1}}(SetProg.Sets.Polar{Float64,SetProg.Sets.EllipsoidAtOrigin{Float64}}(SetProg.Sets.EllipsoidAtOrigin{Float64}([0.00408553 -0.0407022 -0.00047582 -0.00764556; -0.0407022 1.4003 -0.00571605 0.278037; -0.00047582 -0.00571605 0.000936106 -0.00139195; -0.00764556 0.278037 -0.00139195 0.0831464])), [0.88782, 0.00581124, 0.00124382, 0.00892831])

In [90]:
# Load dependencies:
using LinearAlgebra
using SpecialFunctions

# Computed ellipsoid above has:
E = [0.00408553 -0.0407022 -0.00047582 -0.00764556
    -0.0407022 1.4003 -0.00571605 0.278037
    -0.00047582 -0.00571605 0.000936106 -0.00139195
    -0.00764556 0.278037 -0.00139195 0.0831464]

# E = [ 0.00322526  -0.0307753 -0.000335552 -0.003349 
#      -0.0307753    1.37073   -0.0100246    0.216396 
#      -0.000335552 -0.0100246  0.000498443 -0.00201246
#      -0.003349     0.216396  -0.00201246   0.0554606]

singVal = sqrt.(eigvals(E))

volume = pi^(3/2)*prod(singVal)/gamma(3/2+1)

show(volume)

0.0012370726044621107